# Geoindexing

* Imagine you have 1000 points in a 2 dimentional space (latitude, longitude). Lets call these points "data"
* And you are given an extra point. Lets call this point "origin"
* Your task is to find the point from data closest to origin. How would you do this?

## The bad way

* You would calculate the euclidian distiance to every point in data and origin and choose the minimum
* Takes too long. This takes linear time for a search operation relative to number of points in data
* O(n)

## The good way

* Use a spacial index
* Takes logarithmic time
* O(log(n))


# Spacial Indexing

* RTree
* KDTree https://en.wikipedia.org/wiki/K-d_tree

Space partitioning data structure based on n dimentions (in our case 2)

<img src="kdtree.png">